## Paquets i dades originals ## 

In [2]:
import json
import pandas as pd
from datetime import datetime
from listas import disfagia_list, PA_list, P_list, Main_respiratory_infections_list, LRTI_list, COPD_exacerbations_list, Pulmonary_fibrosis_fibrotorax_list
from auxiliary_functions import valores_codigos, contar_diccionarios, dias_ingreso_total, asignar_intervalo_edad, sumar_barthel, sumar_emina, obtener_ultimo_resultat, obtener_valor_promedio, canadenca_comparada, disfagia_mecvvs, extraer_valor_clave, extraer_valor_clave_simple

tabla = pd.read_pickle('./data/processed/dataframe.pkl')
import scipy.stats as stats
from scipy.stats import ttest_ind
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu
from scipy.stats import chi2_contingency


Prèviament, he tractat les dades creant noves columnes que classifiquen millor tota la informació disponible

In [3]:
tabla.head()

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,albumina,proteinas totales,Hb,colesterol total,leucos,limfos,prot C react,urea,FGE MDRD,FGE CDK-EPI
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,None,None,None,None,None,None,None,None,None,None
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,3.1,5.3,14.5,124.7,X,38.9,19.61,25.7,>60,>90
2,96,M,0,[],[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-09 00...",[],...,None,None,None,None,None,None,None,None,None,None
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None
4,93,M,0,"[{'date': '20220208', 'name': 'UREA Sèrum', 'l...","[{'imc': '0', 'anorexia': '1', 'perduaPes': '1...","[{'activitat': '2', 'estatMental': '1', 'humit...",[],"[{'valor': '49.900', 'data': '2022-06-02'}, {'...","[{'codi': 'N02AA', 'dataInici': '2022-02-01 00...","[{'data': '20220330172303', 'disfagiaConeguda'...",...,Proves afegides a un altre petició per extracc...,Proves afegides a un altre petició per extracc...,X,None,X,X,0.54,Proves afegides a un altre petició per extracc...,Proves afegides a un altre petició per extracc...,Proves afegides a un altre petició per extracc...


Distribuiré les dades en 3 grups:
- **CON_PA**: aquest grup està compost per pacients que contenen el codi ICD de pneumònia aspirativa al seu historial.
- **CON_PA_MECVV**: aquest grup conté els pacients que compleixen les següents característiques: 
    - Han donat positiu al MECVV (és a dir, que tenen disfàgia i una alteració en la seguretat o en l'eficàcia)
    - Contenen el codi ICD de pneumònia
    - Han estat ingressats menys de 30 dies en total. 
- **SIN_PA**: aquest grup està format per pacients que contenen el codi ICD de pneumònia o bé:
    - Contenen el codi ICD de pneumònia
    - Han donat positiu al MECVV 
    - Han estat ingressats més de 30 dies en total.

# CON_PA 

In [4]:
CON_PA = tabla[tabla["PA_diagnosticada"] == 1.0]
CON_PA.head(len(tabla))

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,albumina,proteinas totales,Hb,colesterol total,leucos,limfos,prot C react,urea,FGE MDRD,FGE CDK-EPI
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,None,None,None,None,None,None,None,None,None,None
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,3.1,5.3,14.5,124.7,X,38.9,19.61,25.7,>60,>90
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None
6,101,F,0,[],[],"[{'activitat': '3', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'C09AA', 'dataInici': '2021-08-21 00...","[{'data': '20210819124116', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None
12,86,M,0,[],[],"[{'activitat': '1', 'estatMental': '1', 'humit...",[],"[{'valor': '46.4', 'data': '2020-09-09'}]","[{'codi': 'G04CA', 'dataInici': '2020-09-04 00...","[{'data': '20200907110400', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None
13,71,F,0,"[{'date': '20220216', 'name': 'CULTIU Esput', ...",[],"[{'activitat': '3', 'estatMental': '0', 'humit...","[{'dataValoracio': '2020-09-06', 'horaValoraci...",[],"[{'codi': 'C10AA', 'dataInici': '2020-09-06 00...","[{'data': '20200906175506', 'disfagia': 'N', '...",...,4.0,7.4,14.4,80.5,13.95,14.5,6.73,66.0,>60,70
15,75,M,0,[],[],"[{'activitat': '3', 'estatMental': '2', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-09-19 00...",[],...,None,None,None,None,None,None,None,None,None,None
26,97,F,0,"[{'date': '20220419', 'name': 'GLUCOSA Sèrum',...",[],"[{'activitat': '1', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'C09BA', 'dataInici': '2022-04-17 00...",[],...,3.2,6.5,12.0,156.9,14.71,5.0,11.37,119.7,52,48
28,95,F,0,"[{'date': '20220526', 'name': 'GLUCOSA Sèrum',...",[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],"[{'valor': '0', 'data': '2022-05-30'}]","[{'codi': 'C10AA', 'dataInici': '2020-11-18 00...","[{'data': '20220529115841', 'disfagia': 'S', '...",...,3.5,5.8,8.8,186.4,8.31,11.7,2.33,49.9,51,48
29,97,F,0,[],[],"[{'activitat': '3', 'estatMental': '2', 'humit...",[],[],"[{'codi': 'N02BE', 'dataInici': '2020-11-30 00...",[],...,None,None,None,None,None,None,None,None,None,None


# CON_PA_MECVV

In [5]:
CON_PA_MECVV = tabla[
    (tabla["P_diagnosticada"] == 1.0) & (tabla["Dias_totales_ingresado"] < 30) & (tabla["Disfagia_mecvvs"] == 1.0) & (tabla["alteracioEficacia_mecvvs"] == 1.0) |
    (tabla["P_diagnosticada"] == 1.0) & (tabla["Dias_totales_ingresado"] < 30) & (tabla["Disfagia_mecvvs"] == 1.0) & (tabla["alteracioSeguretat_mecvvs"] == 1.0)]   
CON_PA_MECVV.head(len(tabla))

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,albumina,proteinas totales,Hb,colesterol total,leucos,limfos,prot C react,urea,FGE MDRD,FGE CDK-EPI
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None
6,101,F,0,[],[],"[{'activitat': '3', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'C09AA', 'dataInici': '2021-08-21 00...","[{'data': '20210819124116', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None
12,86,M,0,[],[],"[{'activitat': '1', 'estatMental': '1', 'humit...",[],"[{'valor': '46.4', 'data': '2020-09-09'}]","[{'codi': 'G04CA', 'dataInici': '2020-09-04 00...","[{'data': '20200907110400', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None
13,71,F,0,"[{'date': '20220216', 'name': 'CULTIU Esput', ...",[],"[{'activitat': '3', 'estatMental': '0', 'humit...","[{'dataValoracio': '2020-09-06', 'horaValoraci...",[],"[{'codi': 'C10AA', 'dataInici': '2020-09-06 00...","[{'data': '20200906175506', 'disfagia': 'N', '...",...,4.0,7.4,14.4,80.5,13.95,14.5,6.73,66.0,>60,70
27,100,M,0,[],[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],"[{'valor': '0', 'data': '2021-09-14'}, {'valor...","[{'codi': 'C08CA', 'dataInici': '2020-11-23 00...","[{'data': '20201125131827', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None
37,86,M,0,"[{'date': '20221003', 'name': 'GLUCOSA Sèrum',...","[{'imc': '', 'anorexia': '', 'perduaPes': '', ...","[{'activitat': '2', 'estatMental': '1', 'humit...",[],"[{'valor': '72', 'data': '2022-10-09'}]","[{'codi': 'G04CA', 'dataInici': '2022-09-26 00...","[{'data': '20221012122425', 'disfagia': 'S', '...",...,2.9,5.6,13.5,119.5,8.55,9.5,8.16,114.2,None,46
39,87,F,0,"[{'date': '20230619', 'name': 'GLUCOSA Sèrum',...",[],"[{'activitat': '1', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'N02BE', 'dataInici': '2021-01-17 00...","[{'data': '20220111085147', 'disfagia': 'S', '...",...,None,6.2,11.2,None,22.39,X,14.21,30.7,None,>90
41,93,F,0,"[{'date': '20221013', 'name': 'GLUCOSA Sèrum',...",[],"[{'activitat': '1', 'estatMental': '1', 'humit...",[],"[{'valor': '0', 'data': '2023-06-02'}, {'valor...","[{'codi': 'C10AA', 'dataInici': '2021-02-05 00...","[{'data': '20230530155654', 'disfagia': 'S', '...",...,3.5,6.7,13.8,135.7,7.42,27.6,6.43,38.6,>60,48
46,86,M,0,[],[],"[{'activitat': '3', 'estatMental': '1', 'humit...",[],"[{'valor': '55.1', 'data': '2021-03-21'}]","[{'codi': 'G04CA', 'dataInici': '2021-03-20 00...","[{'data': '20210721140136', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None
47,94,M,0,[],[],"[{'activitat': '3', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'C09AA', 'dataInici': '2021-03-23 00...","[{'data': '20210325091551', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None


# SIN_PA

In [6]:
SIN_PA = tabla[
          (tabla["P_diagnosticada"] == 1.0) |
          (tabla["P_diagnosticada"] == 1.0) & (tabla["Disfagia_mecvvs"] == 1.0) & (tabla["alteracioEficacia_mecvvs"] == 1.0) & (tabla["Dias_totales_ingresado"] > 30) |
          (tabla["P_diagnosticada"] == 1.0) & (tabla["Disfagia_mecvvs"] == 1.0) & (tabla["alteracioSeguretat_mecvvs"] == 1.0) & (tabla["Dias_totales_ingresado"] > 30)]
SIN_PA.head(len(tabla))

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,albumina,proteinas totales,Hb,colesterol total,leucos,limfos,prot C react,urea,FGE MDRD,FGE CDK-EPI
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,None,None,None,None,None,None,None,None,None,None
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,3.1,5.3,14.5,124.7,X,38.9,19.61,25.7,>60,>90
2,96,M,0,[],[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-09 00...",[],...,None,None,None,None,None,None,None,None,None,None
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None
4,93,M,0,"[{'date': '20220208', 'name': 'UREA Sèrum', 'l...","[{'imc': '0', 'anorexia': '1', 'perduaPes': '1...","[{'activitat': '2', 'estatMental': '1', 'humit...",[],"[{'valor': '49.900', 'data': '2022-06-02'}, {'...","[{'codi': 'N02AA', 'dataInici': '2022-02-01 00...","[{'data': '20220330172303', 'disfagiaConeguda'...",...,Proves afegides a un altre petició per extracc...,Proves afegides a un altre petició per extracc...,X,None,X,X,0.54,Proves afegides a un altre petició per extracc...,Proves afegides a un altre petició per extracc...,Proves afegides a un altre petició per extracc...
5,57,M,0,"[{'date': '20220909', 'name': 'GLUCOSA Sèrum',...",[],"[{'activitat': '1', 'estatMental': '0', 'humit...",[],"[{'valor': '83.7', 'data': '2020-08-13'}, {'va...","[{'codi': 'N02BE', 'dataInici': '2020-08-12 00...",[],...,4.7,6.3,14.9,186,9.45,33.0,None,30.4,>60,>90
6,101,F,0,[],[],"[{'activitat': '3', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'C09AA', 'dataInici': '2021-08-21 00...","[{'data': '20210819124116', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None
7,90,F,0,"[{'date': '20220627', 'name': 'GLUCOSA Sèrum',...","[{'imc': '0', 'anorexia': '0', 'perduaPes': '1...","[{'activitat': '3', 'estatMental': '1', 'humit...","[{'dataValoracio': '2020-07-30', 'horaValoraci...",[],"[{'codi': 'C09AA', 'dataInici': '2020-07-31 00...","[{'data': '20200731092102', 'disfagia': 'N', '...",...,None,6.9,13.9,None,7.86,40.6,0.32,38.7,None,78
8,90,F,0,"[{'date': '20220827', 'name': 'GLUCOSA Sèrum',...",[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'N02BB', 'dataInici': '2023-07-17 00...",[],...,None,None,13.8,None,5.28,19.1,5.82,35.7,None,74
9,91,M,0,[],[],"[{'activitat': '1', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'C10AA', 'dataInici': '2020-08-25 00...",[],...,None,None,None,None,None,None,None,None,None,None


Ara que tinc totes les dades classificades en aquests 3 grups, començaré a analitzar el seu contingut utilitzant operacions com mitjanes i desviacions estàndard. També compararé els diferents grups amb el pvalor mitjançant T-test, test de Mann-Whitney o test Xi-quadrat. Els criteris que seguiré per utilitzar un test o un altre és fer un test de Shapiro-Wilk per comprovar si segueixen una distribució normal, si és així, usaré el T-test, si no, utilitzaré el de Mann-Whitney. Finalment, si la columna conté valors categòrics com és el cas del sexe (F/M), faré servir el test Xi-quadrat.


## Número

In [7]:
num_CON_PA = len(CON_PA)
num_CON_PA_MECVV= len(CON_PA_MECVV)
num_SIN_PA = len(SIN_PA)

print(num_CON_PA, num_CON_PA_MECVV, num_SIN_PA)

17 10 49


Això també es pot comprovar en observar les files que conté les taules anteriors de cada grup, on veiem que la taula de CON_PA té 17 files, la de CON_PA_MECV 10 i la de SIN_PA 49. 

#### Pvalor

##### Normalitat

In [8]:
stat, p_value = shapiro(range(num_CON_PA))
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9621
Valor p = 0.6704
El nombre de files segueix una distribució normal.


In [9]:
stat, p_value = shapiro(range(num_CON_PA_MECVV))
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9702
Valor p = 0.8924
El nombre de files segueix una distribució normal.


In [10]:
stat, p_value = shapiro(range(num_SIN_PA))
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9556
Valor p = 0.0626
El nombre de files segueix una distribució normal.


In [11]:
t_stat_12, p_val_12 = ttest_ind(CON_PA.index, CON_PA_MECVV.index, equal_var=False)
print(f"P-valor entre CON_PA i CON_PA_MECVV: {p_val_12:.4f}")
alpha = 0.05
if p_value < alpha:
    print("Existeixen diferències significatives entre els 2 grups.")
else:
    print("No existeixen diferències significatives entre els 2 grups.")

P-valor entre CON_PA i CON_PA_MECVV: 0.5246
No existeixen diferències significatives entre els 2 grups.


In [12]:
t_stat_13, p_val_13 = ttest_ind(CON_PA.index, SIN_PA.index, equal_var=False)
print(f"P-valor entre CON_PA i SIN_PA: {p_val_13:.4f}")
alpha = 0.05
if p_value < alpha:
    print("Existeixen diferències significatives entre els 2 grups.")
else:
    print("No existeixen diferències significatives entre els 2 grups.")

P-valor entre CON_PA i SIN_PA: 0.7664
No existeixen diferències significatives entre els 2 grups.


In [13]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV.index, SIN_PA.index, equal_var=False)
print(f"P-valor entre CON_PA_MECVV i SIN_PA: {p_val_12:.4f}")
alpha = 0.05
if p_value < alpha:
    print("Existeixen diferències significatives entre els 2 grups.")
else:
    print("No existeixen diferències significatives entre els 2 grups.")

P-valor entre CON_PA_MECVV i SIN_PA: 0.5246
No existeixen diferències significatives entre els 2 grups.


## Edat

In [14]:
edad_CON_PA = CON_PA['edat'].mean()
desviacion_estandar_edad = CON_PA['edat'].std()

resultado = f"{edad_CON_PA:.2f} ± {desviacion_estandar_edad:.2f}"
print("Mitjana ± Desviació Estàndard de l'edat:", resultado)


Mitjana ± Desviació Estàndard de l'edat: 87.06 ± 8.99


In [15]:
edad_CON_PA_MECVV = CON_PA_MECVV['edat'].mean()
desviacion_estandar_edad = CON_PA_MECVV['edat'].std()

resultado = f"{edad_CON_PA_MECVV:.2f} ± {desviacion_estandar_edad:.2f}"
print("Mitjana ± Desviació Estàndard de l'edat:", resultado)

Mitjana ± Desviació Estàndard de l'edat: 88.90 ± 8.67


In [16]:
edad_SIN_PA = SIN_PA['edat'].mean()
desviacion_estandar_edad = SIN_PA['edat'].std()

resultado = f"{edad_SIN_PA:.2f} ± {desviacion_estandar_edad:.2f}"
print("Mitjana ± Desviació Estàndard de l'edat:", resultado)

Mitjana ± Desviació Estàndard de l'edat: 80.12 ± 17.01


#### Pvalor

##### Normalitat 

In [17]:
stat, p_value = shapiro(CON_PA['edat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9410
Valor p = 0.3297
El nombre de files segueix una distribució normal.


In [18]:
stat, p_value = shapiro(CON_PA_MECVV['edat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9057
Valor p = 0.2528
El nombre de files segueix una distribució normal.


In [19]:
stat, p_value = shapiro(SIN_PA['edat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8539
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [20]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['edat'], CON_PA_MECVV['edat'], equal_var=False)
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: 0.6051223082851969


In [21]:
stat, p_value_13 = mannwhitneyu(CON_PA['edat'], SIN_PA['edat'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_13)

P-valor entre CON_PA i SIN_PA 0.1815160884632735


In [22]:
stat_23, p_value_23 = mannwhitneyu(CON_PA_MECVV['edat'], SIN_PA['edat'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_value_23)

P-valor entre CON_PA_MECVV i SIN_PA: 0.11458529353557328


## Sexe

In [23]:
sexe_CON_PA = CON_PA['sexe'].value_counts()
sexe_CON_PA_MECVV = CON_PA_MECVV['sexe'].value_counts()
sexe_SIN_PA = SIN_PA['sexe'].value_counts()

In [24]:
if 'F' in sexe_CON_PA:
    percent_female1 = (sexe_CON_PA['F'] / len(CON_PA)) * 100
    print(f"Percentatge de sexe femení en CON_PA: {percent_female1:.2f}%")
else:
    print("No hi ha dades de sexe femení en CON_PA")

Percentatge de sexe femení en CON_PA: 47.06%


In [25]:
if 'F' in sexe_CON_PA_MECVV:
    percent_female2 = (sexe_CON_PA_MECVV['F'] / len(CON_PA)) * 100
    print(f"Percentatge de sexe femení en CON_PA_MECVV: {percent_female1:.2f}%")
else:
    print("No hi ha dades de sexe femení en CON_PA_MECVV")

Percentatge de sexe femení en CON_PA_MECVV: 47.06%


In [26]:
if 'F' in sexe_SIN_PA:
    percent_female3 = (sexe_SIN_PA['F'] / len(SIN_PA)) * 100
    print(f"Percentatge de sexe femení en SIN_PA: {percent_female1:.2f}%")
else:
    print("No hi ha dades de sexe femení en SIN_PA")


Percentatge de sexe femení en SIN_PA: 47.06%


#### Pvalor

##### Normalitat

Al ser una variable categòrica no es pot comprovar la seva normalitat. Realitzaré directament el test xi-quadrat

In [27]:
contingency_table_CON_PA_CON_PA_MECVV = pd.crosstab(index=CON_PA['sexe'], columns=CON_PA_MECVV['sexe'])

chi2_CON_PA_CON_PA_MECVV, p_value_CON_PA_CON_PA_MECVV, dof_CON_PA_CON_PA_MECVV, expected_CON_PA_CON_PA_MECVV = chi2_contingency(contingency_table_CON_PA_CON_PA_MECVV)

print("Estadístic xi-quadrat entre CON_PA i CON_PA_MECVV:", chi2_CON_PA_CON_PA_MECVV)
print("P-valor entre CON_PA i CON_PA_MECVV:", p_value_CON_PA_CON_PA_MECVV)

if p_value_CON_PA_CON_PA_MECVV < 0.05:
    print("Hi ha diferències significatives entre les distribucions de sexe en CON_PA i CON_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions de sexe en CON_PA i CON_PA_MECVV.")

Estadístic xi-quadrat entre CON_PA i CON_PA_MECVV: 2.6666666666666665
P-valor entre CON_PA i CON_PA_MECVV: 0.10247043485974942
No hi ha diferències significatives entre les distribucions de sexe en CON_PA i CON_PA_MECVV.


In [28]:
contingency_table_CON_PA_SIN_PA = pd.crosstab(index=CON_PA['sexe'], columns=SIN_PA['sexe'])

chi2_CON_PA_SIN_PA, p_value_CON_PA_SIN_PA, dof_CON_PA_SIN_PA, expected_CON_PA_SIN_PA = chi2_contingency(contingency_table_CON_PA_SIN_PA)

print("Estadístic xi-quadrat entre CON_PA i SIN_PA:", chi2_CON_PA_SIN_PA)
print("P-valor entre CON_PA i SIN_PA:", p_value_CON_PA_SIN_PA)

if p_value_CON_PA_SIN_PA < 0.05:
    print("Hi ha diferències significatives entre les distribucions de sexe en CON_PA i SIN_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de sexe en CON_PA i SIN_PA.")

Estadístic xi-quadrat entre CON_PA i SIN_PA: 13.223042052469136
P-valor entre CON_PA i SIN_PA: 0.00027652843917458906
Hi ha diferències significatives entre les distribucions de sexe en CON_PA i SIN_PA.


In [29]:
contingency_table_CON_PA_MECVV_SIN_PA = pd.crosstab(index=CON_PA_MECVV['sexe'], columns=SIN_PA['sexe'])

chi2_CON_PA_MECVV_SIN_PA, p_value_CON_PA_MECVV_SIN_PA, dof_CON_PA_MECVV_SIN_PA, expected_CON_PA_MECVV_SIN_PA = chi2_contingency(contingency_table_CON_PA_MECVV_SIN_PA)

print("Estadístic xi-quadrat entre CON_PA_MECVV i SIN_PA:", chi2_CON_PA_MECVV_SIN_PA)
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_value_CON_PA_MECVV_SIN_PA)

if p_value_CON_PA_MECVV_SIN_PA < 0.05:
    print("Hi ha diferències significatives entre les distribucions de sexe en CON_PA_MECVV i SIN_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de sexe en CON_PA_MECVV i SIN_PA.")

Estadístic xi-quadrat entre CON_PA_MECVV i SIN_PA: 6.267361111111111
P-valor entre CON_PA_MECVV i SIN_PA: 0.012298217061234292
Hi ha diferències significatives entre les distribucions de sexe en CON_PA_MECVV i SIN_PA.


## Ingressos (admissions)

In [30]:
promedio_ingresos = CON_PA['Num_ingresos'].mean()
desviacion_estandar_ingresos = CON_PA['Num_ingresos'].std()

resultado = f"{promedio_ingresos:.2f} ± {desviacion_estandar_ingresos:.2f}"
print("Mitjana ± Desviació Estàndard del nombre d'ingressos:", resultado)

Mitjana ± Desviació Estàndard del nombre d'ingressos: 5.59 ± 3.37


In [31]:
promedio_ingresos = CON_PA_MECVV['Num_ingresos'].mean()
desviacion_estandar_ingresos = CON_PA_MECVV['Num_ingresos'].std()

resultado = f"{promedio_ingresos:.2f} ± {desviacion_estandar_ingresos:.2f}"
print("Mitjana ± Desviació Estàndard del nombre d'ingressos:", resultado)

Mitjana ± Desviació Estàndard del nombre d'ingressos: 5.90 ± 2.08


In [32]:
promedio_ingresos = SIN_PA['Num_ingresos'].mean()
desviacion_estandar_ingresos = SIN_PA['Num_ingresos'].std()

resultado = f"{promedio_ingresos:.2f} ± {desviacion_estandar_ingresos:.2f}"
print("Mitjana ± Desviació Estàndard del nombre d'ingressos:", resultado)

Mitjana ± Desviació Estàndard del nombre d'ingressos: 6.69 ± 5.13


#### Pvalor

##### Normalitat

In [33]:
stat, p_value = shapiro(CON_PA['Num_ingresos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8008
Valor p = 0.0021
El nombre de files no segueix una distribució normal.


In [34]:
stat, p_value = shapiro(CON_PA_MECVV['Num_ingresos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9558
Valor p = 0.7376
El nombre de files segueix una distribució normal.


In [35]:
stat, p_value = shapiro(SIN_PA['Num_ingresos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.7277
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [36]:
stat, p_value_12 = mannwhitneyu(CON_PA['Num_ingresos'], CON_PA_MECVV['Num_ingresos'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA i CON_PA_MECVV 0.3599581940859352


In [37]:
stat, p_value_13 = mannwhitneyu(CON_PA['Num_ingresos'], SIN_PA['Num_ingresos'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_13)

P-valor entre CON_PA i SIN_PA 0.514931142191354


In [38]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['Num_ingresos'], SIN_PA['Num_ingresos'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV i SIN_PA 0.7831397976578486


## Disfàgia

In [39]:
disf_CON_PA= (CON_PA['DO_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de disfàgia: {disf_CON_PA:.2f}%")

Percentatge de disfàgia: 47.06%


In [40]:
disf_CON_PA_MECVV = (CON_PA_MECVV['DO_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de disfàgia: {disf_CON_PA_MECVV:.2f}%")

Percentatge de disfàgia: 30.00%


In [41]:
disf_SIN_PA= (SIN_PA['DO_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de disfàgia: {disf_SIN_PA:.2f}%")

Percentatge de disfàgia: 18.37%


#### Pvalor

##### Normalitat

In [42]:
stat, p_value = shapiro(CON_PA['DO_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.6418
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [43]:
stat, p_value = shapiro(CON_PA_MECVV['DO_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.5942
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [44]:
stat, p_value = shapiro(SIN_PA['DO_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.4716
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [45]:
stat, p_value_12 = mannwhitneyu(CON_PA['DO_diagnosticada'], CON_PA_MECVV['DO_diagnosticada'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA i CON_PA_MECVV 0.40914513229166993


In [46]:
stat, p_value_13 = mannwhitneyu(CON_PA['DO_diagnosticada'], SIN_PA['DO_diagnosticada'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_13)

P-valor entre CON_PA i SIN_PA 0.02125060274444335


In [47]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['DO_diagnosticada'], SIN_PA['DO_diagnosticada'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV i SIN_PA 0.4172116903428992


## Index de Barthel

### Promedio

In [48]:
promedio_barthel = CON_PA['Barthel_resultados'].mean()
desviacion_estandar_barthel = CON_PA['Barthel_resultados'].std()

resultado = f"{promedio_barthel:.2f} ± {desviacion_estandar_barthel:.2f}"
print("Mitjana ± Desviació Estàndard del Barthel:", resultado)

Mitjana ± Desviació Estàndard del Barthel: 22.65 ± 24.63


In [49]:
promedio_barthel = CON_PA_MECVV['Barthel_resultados'].mean()
desviacion_estandar_barthel = CON_PA_MECVV['Barthel_resultados'].std()

resultado = f"{promedio_barthel:.2f} ± {desviacion_estandar_barthel:.2f}"
print("Mitjana ± Desviació Estàndard del Barthel:", resultado)

Mitjana ± Desviació Estàndard del Barthel: 28.50 ± 22.24


In [50]:
promedio_barthel = SIN_PA['Barthel_resultados'].mean()
desviacion_estandar_barthel = SIN_PA['Barthel_resultados'].std()

resultado = f"{promedio_barthel:.2f} ± {desviacion_estandar_barthel:.2f}"
print("Mitjana ± Desviació Estàndard del Barthel:", resultado)

Mitjana ± Desviació Estàndard del Barthel: 48.88 ± 37.42


#### Pvalor

##### Normalitat

In [51]:
stat, p_value = shapiro(CON_PA['Barthel_resultados'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8517
Valor p = 0.0114
El nombre de files no segueix una distribució normal.


In [52]:
stat, p_value = shapiro(CON_PA_MECVV['Barthel_resultados'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9138
Valor p = 0.3083
El nombre de files segueix una distribució normal.


In [53]:
stat, p_value = shapiro(SIN_PA['Barthel_resultados'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8858
Valor p = 0.0002
El nombre de files no segueix una distribució normal.


In [54]:
stat, p_value_12 = mannwhitneyu(CON_PA['Barthel_resultados'], CON_PA_MECVV['Barthel_resultados'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA i CON_PA_MECVV 0.41631085084208264


In [55]:
stat, p_value_13 = mannwhitneyu(CON_PA['Barthel_resultados'], SIN_PA['Barthel_resultados'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_13)

P-valor entre CON_PA i SIN_PA 0.012373091641540783


In [56]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['Barthel_resultados'], SIN_PA['Barthel_resultados'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV i SIN_PA 0.13803563369050034


### Valors de dependència

##### CON_PA

In [57]:
indepe_CON_PA = (CON_PA['Barthel_resultados'] == 100.0).mean() * 100
print(f"Percentatge de indepe_CON_PA: {indepe_CON_PA:.2f}%")

Percentatge de indepe_CON_PA: 0.00%


In [58]:
mod_CON_PA = (CON_PA['Barthel_resultados'].between(61, 95)).mean() * 100
print(f"Percentatge de mod_CON_PA: {mod_CON_PA:.2f}%")

Percentatge de mod_CON_PA: 5.88%


In [59]:
sev_CON_PA = (CON_PA['Barthel_resultados'].between(21, 60)).mean() * 100
print(f"Percentatge de sev_CON_PA: {sev_CON_PA:.2f}%")

Percentatge de sev_CON_PA: 41.18%


In [60]:
total_CON_PA = (CON_PA['Barthel_resultados'] <= 20.0 ).mean() * 100
print(f"Percentatge de total_CON_PA: {total_CON_PA:.2f}%")

Percentatge de total_CON_PA: 52.94%


##### CON_PA_MECVV

In [61]:
indepe_CON_PA_MECVV = (CON_PA_MECVV['Barthel_resultados'] == 100.0).mean() * 100
print(f"Percentatge de indepe_CON_PA_MECVV: {indepe_CON_PA_MECVV:.2f}%")

Percentatge de indepe_CON_PA_MECVV: 0.00%


In [62]:
mod_CON_PA_MECVV = (CON_PA_MECVV['Barthel_resultados'].between(61,95)).mean() * 100
print(f"Percentatge de mod_CON_PA_MECVV: {mod_CON_PA_MECVV:.2f}%")

Percentatge de mod_CON_PA_MECVV: 0.00%


In [63]:
sev_CON_PA_MECVV = (CON_PA_MECVV['Barthel_resultados'].between(21,60)).mean() * 100
print(f"Percentatge de sev_CON_PA_MECVV: {sev_CON_PA_MECVV:.2f}%")

Percentatge de sev_CON_PA_MECVV: 60.00%


In [64]:
total_CON_PA_MECVV = (CON_PA_MECVV['Barthel_resultados'] <= 20.0).mean() * 100
print(f"Percentatge de total_CON_PA_MECVV: {total_CON_PA_MECVV:.2f}%")

Percentatge de total_CON_PA_MECVV: 40.00%


##### SIN_PA

In [65]:
indepe_SIN_PA = (SIN_PA['Barthel_resultados'] == 100.0).mean() * 100
print(f"Percentatge de indepe_SIN_PA: {indepe_SIN_PA:.2f}%")

Percentatge de indepe_SIN_PA: 20.41%


In [66]:
mod_SIN_PA = (SIN_PA['Barthel_resultados'].between(61,95)).mean() * 100
print(f"Percentatge de mod_SIN_PA: {mod_SIN_PA:.2f}%")

Percentatge de mod_SIN_PA: 18.37%


In [67]:
sev_SIN_PA = (SIN_PA['Barthel_resultados'].between(21,60)).mean() * 100
print(f"Percentatge de sev_SIN_PA: {sev_SIN_PA:.2f}%")

Percentatge de sev_SIN_PA: 28.57%


In [68]:
total_SIN_PA = (SIN_PA['Barthel_resultados'] <= 20).mean() * 100
print(f"Percentatge de total_SIN_PA: {total_SIN_PA:.2f}%")

Percentatge de total_SIN_PA: 32.65%


#### Pvalor

##### Independència

In [69]:
stat_12, p_value_12 = mannwhitneyu(CON_PA['Barthel_resultados'] == 100.0, CON_PA_MECVV['Barthel_resultados'] == 100.0, alternative='two-sided')
print("P-valor entre indepe_CON_PA i indepe_CON_PA_MECVV:", p_value_12)

P-valor entre indepe_CON_PA i indepe_CON_PA_MECVV: 1.0


In [70]:
stat_13, p_value_13 = mannwhitneyu(CON_PA['Barthel_resultados'] == 100.0, SIN_PA['Barthel_resultados'] == 100.0, alternative='two-sided')
print("P-valor entre indepe_CON_PA i indepe_SIN_PA:", p_value_13)

P-valor entre indepe_CON_PA i indepe_SIN_PA: 0.04605091215969595


In [71]:
stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['Barthel_resultados'] == 100.0, SIN_PA['Barthel_resultados'] == 100.0, equal_var=False)
print("P-valor entre indepe_CON_PA_MECVV i SIN_PA:", p_value_23)

P-valor entre indepe_CON_PA_MECVV i SIN_PA: 0.13803563369050034


##### Moderada

In [72]:
stat_12, p_value_12 = mannwhitneyu(CON_PA['Barthel_resultados'].between(61, 95), CON_PA_MECVV['Barthel_resultados'].between(61, 95), alternative='two-sided')
print("P-valor entre mod_CON_PA i mod_CON_PA_MECVV:", p_value_12)

P-valor entre mod_CON_PA i mod_CON_PA_MECVV: 0.49002535950179893


In [73]:
stat_13, p_value_13 = mannwhitneyu(CON_PA['Barthel_resultados'].between(61, 95), SIN_PA['Barthel_resultados'].between(61, 95), alternative='two-sided')
print("P-valor entre indepe_CON_PA i mod_SIN_PA:", p_value_13)

P-valor entre indepe_CON_PA i mod_SIN_PA: 0.22404485220846948


In [74]:
stat_23, p_value_23 = mannwhitneyu(CON_PA_MECVV['Barthel_resultados'].between(61, 95), SIN_PA['Barthel_resultados'].between(61, 95), alternative='two-sided')  
print("P-valor entre mod_CON_PA_MECVV i mod_SIN_PA :", p_value_23)

P-valor entre mod_CON_PA_MECVV i mod_SIN_PA : 0.14889565499123467


##### Severa

In [75]:
stat_12, p_value_12 = mannwhitneyu(CON_PA['Barthel_resultados'].between(21,60), CON_PA_MECVV['Barthel_resultados'].between(21,60), alternative='two-sided')
print("P-valor entre sev_CON_PA i sev_CON_PA_MECVV:", p_value_12)

P-valor entre sev_CON_PA i sev_CON_PA_MECVV: 0.3688417252786701


In [76]:
stat_13, p_value_13 = mannwhitneyu(CON_PA['Barthel_resultados'].between(21,60), SIN_PA['Barthel_resultados'].between(21,60), alternative='two-sided')
print("P-valor entre sev_CON_PA i sev_SIN_PA:", p_value_13)

P-valor entre sev_CON_PA i sev_SIN_PA: 0.3446368758082651


In [77]:
t_stat_23, p_value_23 = mannwhitneyu(CON_PA_MECVV['Barthel_resultados'].between(21,60), SIN_PA['Barthel_resultados'].between(21,60), alternative='two-sided' )
print("P-valor entre sev_CON_PA_MECVV i sev_SIN_PA:", p_value_23)

P-valor entre sev_CON_PA_MECVV i sev_SIN_PA: 0.05945988532601827


##### Total

In [78]:
stat_12, p_value_12 = mannwhitneyu(CON_PA['Barthel_resultados'] <= 20.0, CON_PA_MECVV['Barthel_resultados'] <= 20.0, alternative='two-sided')
print("P-valor entre total_CON_PA i total_CON_PA_MECVV:", p_value_12)

P-valor entre total_CON_PA i total_CON_PA_MECVV: 0.5426838040005246


In [79]:
stat_13, p_value_13 = mannwhitneyu(CON_PA['Barthel_resultados'] <= 20.0, SIN_PA['Barthel_resultados'] <= 20.0, alternative='two-sided')
print("P-valor entre total_CON_PA i total_SIN_PA:", p_value_13)

P-valor entre total_CON_PA i total_SIN_PA: 0.14269651822055499


In [80]:
stat_23, p_value_23 = mannwhitneyu(CON_PA_MECVV['Barthel_resultados'] <= 20.0, SIN_PA['Barthel_resultados'] <= 20.0, alternative='two-sided')
print("P-valor entre total_CON_PA_MECVV i total_SIN_PA:", p_value_23)

P-valor entre total_CON_PA_MECVV i total_SIN_PA: 0.6663533091466043


## Index del MNA

Primer convertiré la columna a tipus numèric ja que els valors que conté són de tipus objecte i amb aquest tipus no puc calcular la mitjana ni la desviació estàndard.

In [81]:
CON_PA.loc[:, 'mna_resultats'] = pd.to_numeric(CON_PA['mna_resultats'], errors='coerce')
CON_PA_MECVV.loc[:, 'mna_resultats'] = pd.to_numeric(CON_PA_MECVV['mna_resultats'], errors='coerce')
SIN_PA.loc[:, 'mna_resultats'] = pd.to_numeric(SIN_PA['mna_resultats'], errors='coerce')

### Mitjana

In [82]:
promedio_mna = CON_PA['mna_resultats'].mean()
desviacion_estandar_mna = CON_PA['mna_resultats'].std()

resultado = f"{promedio_mna:.2f} ± {desviacion_estandar_mna:.2f}"
print("Mitjana ± Desviació Estándard del mna:", resultado)

Mitjana ± Desviació Estándard del mna: 0.00 ± 0.00


In [83]:
promedio_mna = CON_PA_MECVV['mna_resultats'].mean()
desviacion_estandar_mna = CON_PA_MECVV['mna_resultats'].std()

resultado = f"{promedio_mna:.2f} ± {desviacion_estandar_mna:.2f}"
print("Mitjana ± Desviació Estándard del mna:", resultado)

Mitjana ± Desviació Estándard del mna: 0.00 ± nan


In [84]:
promedio_mna = SIN_PA['mna_resultats'].mean()
desviacion_estandar_mna = SIN_PA['mna_resultats'].std()

resultado = f"{promedio_mna:.2f} ± {desviacion_estandar_mna:.2f}"
print("Mitjana ± Desviació Estàndard del mna:", resultado)

Mitjana ± Desviació Estàndard del mna: 3.05 ± 5.92


#### Pvalor

##### Normalitat

In [85]:
conpanone = CON_PA[CON_PA['mna_resultats'].notna()]
stat, p_value = shapiro(conpanone)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [86]:
pa2none = CON_PA_MECVV[CON_PA_MECVV['mna_resultats'].notna()]
stat, p_value = shapiro(pa2none)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [87]:
pa3none = SIN_PA[SIN_PA['mna_resultats'].notna()]
stat, p_value = shapiro(pa3none)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [88]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['mna_resultats'], CON_PA_MECVV['mna_resultats'], equal_var=False)
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: nan


In [89]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['mna_resultats'], SIN_PA['mna_resultats'], equal_var=False)
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: nan


In [90]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['mna_resultats'], SIN_PA['mna_resultats'], equal_var=False)
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: nan


### Valors de malnutrició

##### CON_PA

In [91]:
norm_CON_PA = CON_PA['mna_resultats'].between(24, 30)
Percentatge_norm_CON_PA = norm_CON_PA.mean() * 100
num_observaciones_norm_CON_PA = norm_CON_PA.sum()

print(f"Percentatge de norm_CON_PA: {Percentatge_norm_CON_PA:.2f}%")
print(f"Nombre d'observacions en norm_CON_PA: {num_observaciones_norm_CON_PA}")

Percentatge de norm_CON_PA: 0.00%
Nombre d'observacions en norm_CON_PA: 0


In [92]:
malnutr_CON_PA = CON_PA['mna_resultats'].between(17, 23.5)
Percentatge_malnutr_CON_PA = malnutr_CON_PA.mean() * 100
num_observaciones_malnutr_CON_PA = norm_CON_PA.sum()

print(f"Percentatge de malnutr_CON_PA: {Percentatge_malnutr_CON_PA:.2f}%")
print(f"Nombre d'observacions en norm_CON_PA: {num_observaciones_malnutr_CON_PA}")

Percentatge de malnutr_CON_PA: 0.00%
Nombre d'observacions en norm_CON_PA: 0


In [93]:
malnut_CON_PA = CON_PA['mna_resultats'] < 17
Percentatge_malnut_CON_PA = malnut_CON_PA.mean() * 100
num_observaciones_malnut_CON_PA = malnut_CON_PA.sum()

print(f"Percentatge de malnut_CON_PA: {Percentatge_malnut_CON_PA:.2f}%")
print(f"Nombre d'observacions en malnut_CON_PA: {num_observaciones_malnut_CON_PA}")

Percentatge de malnut_CON_PA: 23.53%
Nombre d'observacions en malnut_CON_PA: 4


##### CON_PA_MECVV

In [94]:
norm_CON_PA_MECVV = CON_PA_MECVV['mna_resultats'].between(24, 30)
Percentatge_norm_CON_PA_MECVV = norm_CON_PA_MECVV.mean() * 100
num_observaciones_norm_CON_PA_MECVV = norm_CON_PA_MECVV.sum()

print(f"Percentatge de norm_CON_PA_MECVV: {Percentatge_norm_CON_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en norm_CON_PA_MECVV: {num_observaciones_norm_CON_PA_MECVV}")

Percentatge de norm_CON_PA_MECVV: 0.00%
Nombre d'observacions en norm_CON_PA_MECVV: 0


In [95]:
malnutr_CON_PA_MECVV = CON_PA_MECVV['mna_resultats'].between(17, 23.5)
Percentatge_malnutr_CON_PA_MECVV = malnutr_CON_PA_MECVV.mean() * 100
num_observaciones_malnutr_CON_PA_MECVV = norm_CON_PA_MECVV.sum()

print(f"Percentatge de malnutr_CON_PA_MECVV: {Percentatge_malnutr_CON_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en norm_CON_PA_MECVV: {num_observaciones_malnutr_CON_PA_MECVV}")

Percentatge de malnutr_CON_PA_MECVV: 0.00%
Nombre d'observacions en norm_CON_PA_MECVV: 0


In [96]:
malnut_CON_PA_MECVV = CON_PA_MECVV['mna_resultats'] < 17
Percentatge_malnut_CON_PA_MECVV = malnut_CON_PA_MECVV.mean() * 100
num_observaciones_malnut_CON_PA_MECVV = malnut_CON_PA_MECVV.sum()

print(f"Percentatge de malnut_CON_PA_MECVV: {Percentatge_malnut_CON_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en malnut_CON_PA_MECVV: {num_observaciones_malnut_CON_PA_MECVV}")

Percentatge de malnut_CON_PA_MECVV: 10.00%
Nombre d'observacions en malnut_CON_PA_MECVV: 1


##### SIN_PA

In [97]:
norm_SIN_PA = SIN_PA['mna_resultats'].between(24, 30)
Percentatge_norm_SIN_PA = norm_SIN_PA.mean() * 100
num_observaciones_norm_SIN_PA = norm_SIN_PA.sum()

print(f"Percentatge de norm_SIN_PA: {Percentatge_norm_SIN_PA:.2f}%")
print(f"Nombre d'observacions en norm_SIN_PA: {num_observaciones_norm_SIN_PA}")

Percentatge de norm_SIN_PA: 0.00%
Nombre d'observacions en norm_SIN_PA: 0


In [98]:
malnutr_SIN_PA = SIN_PA['mna_resultats'].between(17, 23.5)
Percentatge_malnutr_SIN_PA = malnutr_SIN_PA.mean() * 100
num_observaciones_malnutr_SIN_PA = malnutr_SIN_PA.sum()

print(f"Percentatge de malnutr_SIN_PA: {Percentatge_malnutr_SIN_PA:.2f}%")
print(f"Nombre d'observacions en malnutr_SIN_PA: {num_observaciones_malnutr_SIN_PA}")

Percentatge de malnutr_SIN_PA: 2.04%
Nombre d'observacions en malnutr_SIN_PA: 1


In [99]:
malnut_SIN_PA = SIN_PA['mna_resultats'] < 17
Percentatge_malnut_SIN_PA = malnut_SIN_PA.mean() * 100
num_observaciones_malnut_SIN_PA = malnut_SIN_PA.sum()

print(f"Percentatge de malnut_SIN_PA: {Percentatge_malnut_SIN_PA:.2f}%")
print(f"Nombre d'observacions en malnut_SIN_PA: {num_observaciones_malnut_SIN_PA}")

Percentatge de malnut_SIN_PA: 18.37%
Nombre d'observacions en malnut_SIN_PA: 9


#### Pvalor

##### Normal

In [100]:
t_stat_12, p_val_12 = mannwhitneyu(CON_PA['mna_resultats'].between(24, 30), CON_PA_MECVV['mna_resultats'].between(24, 30), alternative='two-sided')
print("P-valor entre norm_CON_PA i norm_CON_PA_MECVV:", p_val_12)

P-valor entre norm_CON_PA i norm_CON_PA_MECVV: 1.0


In [101]:
t_stat_13, p_val_13 = mannwhitneyu(CON_PA['mna_resultats'].between(24, 30), SIN_PA['mna_resultats'].between(24, 30), alternative='two-sided')
print("P-valor entre norm_CON_PA i norm_SIN_PA:", p_val_13)

P-valor entre norm_CON_PA i norm_SIN_PA: 1.0


In [102]:
t_stat_23, p_val_23 = mannwhitneyu(CON_PA_MECVV['mna_resultats'].between(24, 30), SIN_PA['mna_resultats'].between(24, 30), alternative='two-sided')
print("P-valor entre norm_CON_PA_MECVV i norm_SIN_PA:", p_val_23)

P-valor entre norm_CON_PA_MECVV i norm_SIN_PA: 1.0


##### Risc de malnutrició

In [103]:
t_stat_12, p_val_12 = mannwhitneyu(CON_PA['mna_resultats'].between(17, 23.5), CON_PA_MECVV['mna_resultats'].between(17, 23.5), alternative='two-sided')
print("P-valor entre malnutr_CON_PA i malnutr_CON_PA_MECVV:", p_val_12)

P-valor entre malnutr_CON_PA i malnutr_CON_PA_MECVV: 1.0


In [104]:
t_stat_13, p_val_13 = mannwhitneyu(CON_PA['mna_resultats'].between(17, 23.5), SIN_PA['mna_resultats'].between(17, 23.5), alternative='two-sided')
print("P-valor entre malnutr_CON_PA i malnutr_SIN_PA:", p_val_13)

P-valor entre malnutr_CON_PA i malnutr_SIN_PA: 0.5793276078637376


In [105]:
t_stat_23, p_val_23 = mannwhitneyu(CON_PA_MECVV['mna_resultats'].between(17, 23.5), SIN_PA['mna_resultats'].between(17, 23.5), alternative='two-sided')
print("P-valor entre malnutr_CON_PA_MECVV i malnutr_SIN_PA:", p_val_23)

P-valor entre malnutr_CON_PA_MECVV i malnutr_SIN_PA: 0.6843175494727258


##### Malnutrició

In [106]:
t_stat_12, p_val_12 = mannwhitneyu(CON_PA['mna_resultats'] <17 , 
                                CON_PA_MECVV['mna_resultats'] < 17, alternative='two-sided')
print("P-valor entre malnut_CON_PA i norm_CON_PA_MECVV:", p_val_12)

P-valor entre malnut_CON_PA i norm_CON_PA_MECVV: 0.4120280546315189


In [107]:
t_stat_13, p_val_13 = mannwhitneyu(CON_PA['mna_resultats'] <17 , 
                                SIN_PA['mna_resultats'] < 17, alternative='two-sided')
print("P-valor entre malnut_CON_PA i malnut_SIN_PA:", p_val_13)

P-valor entre malnut_CON_PA i malnut_SIN_PA: 0.6549000779898075


In [108]:
t_stat_23, p_val_23 = mannwhitneyu(CON_PA_MECVV['mna_resultats'] <17 , 
                                SIN_PA['mna_resultats'] < 17, alternative='two-sided')
print("P-valor entre malnut_CON_PA_MECVV i malnut_SIN_PA:", p_val_23)

P-valor entre malnut_CON_PA_MECVV i malnut_SIN_PA: 0.5341419377112933


## Index de l'EMINA

### Mitjana

In [109]:
promedio_emina = CON_PA['EMINA_sumatorios_comparados'].mean()
desviacion_estandar_emina = CON_PA['EMINA_sumatorios_comparados'].std()

resultado = f"{promedio_emina:.2f} ± {desviacion_estandar_emina:.2f}"
print("Mitjana ± Desviació Estàndard de l'emina:", resultado)

Mitjana ± Desviació Estàndard de l'emina: 7.94 ± 3.21


In [110]:
promedio_emina = CON_PA_MECVV['EMINA_sumatorios_comparados'].mean()
desviacion_estandar_emina = CON_PA_MECVV['EMINA_sumatorios_comparados'].std()

resultado = f"{promedio_emina:.2f} ± {desviacion_estandar_emina:.2f}"
print("Mitjana ± Desviació Estàndard de l'emina:", resultado)

Mitjana ± Desviació Estàndard de l'emina: 7.60 ± 2.01


In [111]:
promedio_emina = SIN_PA['EMINA_sumatorios_comparados'].mean()
desviacion_estandar_emina = SIN_PA['EMINA_sumatorios_comparados'].std()

resultado = f"{promedio_emina:.2f} ± {desviacion_estandar_emina:.2f}"
print("Mitjana ± Desviació Estàndard de l'emina:", resultado)

Mitjana ± Desviació Estàndard de l'emina: 5.64 ± 4.08


#### Pvalor

##### Normalitat

In [112]:
stat, p_value = shapiro(CON_PA['EMINA_sumatorios_comparados']).dropna()
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

AttributeError: 'ShapiroResult' object has no attribute 'dropna'

In [113]:
stat, p_value = shapiro(CON_PA_MECVV['EMINA_sumatorios_comparados']).dropna()
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

AttributeError: 'ShapiroResult' object has no attribute 'dropna'

In [114]:
stat, p_value = shapiro(SIN_PA['EMINA_sumatorios_comparados']).dropna()
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

AttributeError: 'ShapiroResult' object has no attribute 'dropna'

In [115]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['EMINA_sumatorios_comparados'], CON_PA_MECVV['EMINA_sumatorios_comparados'], equal_var=False)
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: 0.7372065144531302


In [116]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['EMINA_sumatorios_comparados'], SIN_PA['EMINA_sumatorios_comparados'], equal_var=False)
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: nan


In [117]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['EMINA_sumatorios_comparados'], SIN_PA['EMINA_sumatorios_comparados'], equal_var=False)
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: nan


### Valors EMINA

##### CON_PA

In [118]:
lowr_CON_PA = CON_PA['EMINA_sumatorios_comparados'].between(0, 5)
Percentatge_lowr_CON_PA = lowr_CON_PA.mean() * 100
num_observaciones_lowr_CON_PA = lowr_CON_PA.sum()

print(f"Percentatge de lowr_CON_PA: {Percentatge_lowr_CON_PA:.2f}%")
print(f"Nombre d'observacions en lowr_CON_PA: {num_observaciones_lowr_CON_PA}")

Percentatge de lowr_CON_PA: 17.65%
Nombre d'observacions en lowr_CON_PA: 3


In [119]:
modr_CON_PA = CON_PA['EMINA_sumatorios_comparados'].between(6, 10)
Percentatge_modr_CON_PA = modr_CON_PA.mean() * 100
num_observaciones_modr_CON_PA = modr_CON_PA.sum()

print(f"Percentatge de modr_CON_PA: {Percentatge_modr_CON_PA:.2f}%")
print(f"Nombre d'observacions en modr_CON_PA: {num_observaciones_modr_CON_PA}")

Percentatge de modr_CON_PA: 64.71%
Nombre d'observacions en modr_CON_PA: 11


In [120]:
highr_CON_PA = CON_PA['EMINA_sumatorios_comparados'].between(11, 15)
Percentatge_highr_CON_PA = highr_CON_PA.mean() * 100
num_observaciones_highr_CON_PA = highr_CON_PA.sum()

print(f"Percentatge de highr_CON_PA: {Percentatge_highr_CON_PA:.2f}%")
print(f"Nombre d'observacions en highr_CON_PA: {num_observaciones_highr_CON_PA}")

Percentatge de highr_CON_PA: 17.65%
Nombre d'observacions en highr_CON_PA: 3


##### CON_PA_MECVV

In [121]:
lowr_CON_PA_MECVV = CON_PA_MECVV['EMINA_sumatorios_comparados'].between(0, 5)
Percentatge_lowr_CON_PA_MECVV = lowr_CON_PA_MECVV.mean() * 100
num_observaciones_lowr_CON_PA_MECVV = lowr_CON_PA_MECVV.sum()

print(f"Percentatge de lowr_CON_PA_MECVV: {Percentatge_lowr_CON_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en lowr_CON_PA_MECVV: {num_observaciones_lowr_CON_PA_MECVV}")

Percentatge de lowr_CON_PA_MECVV: 20.00%
Nombre d'observacions en lowr_CON_PA_MECVV: 2


In [122]:
modr_CON_PA_MECVV = CON_PA_MECVV['EMINA_sumatorios_comparados'].between(6, 10)
Percentatge_modr_CON_PA_MECVV = modr_CON_PA_MECVV.mean() * 100
num_observaciones_modr_CON_PA_MECVV = modr_CON_PA_MECVV.sum()

print(f"Percentatge de modr_CON_PA_MECVV: {Percentatge_modr_CON_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en modr_CON_PA_MECVV: {num_observaciones_modr_CON_PA_MECVV}")

Percentatge de modr_CON_PA_MECVV: 80.00%
Nombre d'observacions en modr_CON_PA_MECVV: 8


In [123]:
highr_CON_PA_MECVV = CON_PA_MECVV['EMINA_sumatorios_comparados'].between(11, 15)
Percentatge_highr_CON_PA_MECVV = highr_CON_PA_MECVV.mean() * 100
num_observaciones_highr_CON_PA_MECVV = highr_CON_PA_MECVV.sum()

print(f"Percentatge de highr_CON_PA_MECVV: {Percentatge_highr_CON_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en highr_CON_PA_MECVV: {num_observaciones_highr_CON_PA_MECVV}")

Percentatge de highr_CON_PA_MECVV: 0.00%
Nombre d'observacions en highr_CON_PA_MECVV: 0


##### SIN_PA

In [124]:
lowr_SIN_PA = SIN_PA['EMINA_sumatorios_comparados'].between(0, 5)
Percentatge_lowr_SIN_PA = lowr_SIN_PA.mean() * 100
num_observaciones_lowr_SIN_PA = lowr_SIN_PA.sum()

print(f"Percentatge de lowr_SIN_PA: {Percentatge_lowr_SIN_PA:.2f}%")
print(f"Nombre d'observacions en lowr_SIN_PA: {num_observaciones_lowr_SIN_PA}")

Percentatge de lowr_SIN_PA: 44.90%
Nombre d'observacions en lowr_SIN_PA: 22


In [125]:
modr_SIN_PA = SIN_PA['EMINA_sumatorios_comparados'].between(6, 10)
Percentatge_modr_SIN_PA = modr_SIN_PA.mean() * 100
num_observaciones_modr_SIN_PA = modr_SIN_PA.sum()

print(f"Percentatge de modr_SIN_PA: {Percentatge_modr_SIN_PA:.2f}%")
print(f"Nombre d'observacions en modr_SIN_PA: {num_observaciones_modr_SIN_PA}")

Percentatge de modr_SIN_PA: 40.82%
Nombre d'observacions en modr_SIN_PA: 20


In [126]:
highr_SIN_PA = SIN_PA['EMINA_sumatorios_comparados'].between(11, 15)
Percentatge_highr_SIN_PA = highr_SIN_PA.mean() * 100
num_observaciones_highr_SIN_PA = highr_SIN_PA.sum()

print(f"Percentatge de highr_SIN_PA: {Percentatge_highr_SIN_PA:.2f}%")
print(f"Nombre d'observacions en highr_SIN_PA: {num_observaciones_highr_SIN_PA}")

Percentatge de highr_SIN_PA: 10.20%
Nombre d'observacions en highr_SIN_PA: 5


#### Pvalor 

##### Risc baix

In [127]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['EMINA_sumatorios_comparados'].between(0,5), CON_PA_MECVV['EMINA_sumatorios_comparados'].between(0,5), equal_var=False)
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: 0.8874457485314164


In [128]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['EMINA_sumatorios_comparados'].between(0,5), SIN_PA['EMINA_sumatorios_comparados'].between(0,5), equal_var=False)
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: 0.028470032037651503


In [129]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['EMINA_sumatorios_comparados'].between(0,5), SIN_PA['EMINA_sumatorios_comparados'].between(0,5), equal_var=False)
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: 0.12129243251471264


##### Risc moderat

In [130]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['EMINA_sumatorios_comparados'].between(6,10), CON_PA_MECVV['EMINA_sumatorios_comparados'].between(6,10), equal_var=False)
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: 0.40237952012988454


In [131]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['EMINA_sumatorios_comparados'].between(6,10), SIN_PA['EMINA_sumatorios_comparados'].between(6,10), equal_var=False)
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: 0.09654829012396034


In [132]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['EMINA_sumatorios_comparados'].between(6,10), SIN_PA['EMINA_sumatorios_comparados'].between(6,10), equal_var=False)
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: 0.02066325652755892


##### Risc alt

In [133]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['EMINA_sumatorios_comparados'].between(11,15), CON_PA_MECVV['EMINA_sumatorios_comparados'].between(11,15), equal_var=False)
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: 0.08262239198015026


In [134]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['EMINA_sumatorios_comparados'].between(11,15), SIN_PA['EMINA_sumatorios_comparados'].between(11,15), equal_var=False)
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: 0.48486212369214565


In [135]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['EMINA_sumatorios_comparados'].between(11,15), SIN_PA['EMINA_sumatorios_comparados'].between(11,15), equal_var=False)
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: 0.02374418239053046


## Index de la canadenca 

### Mitjana

In [136]:
promedio_canad = CON_PA['Canadenca_sumatorios_comparados'].mean()
desviacion_estandar_canad = CON_PA['Canadenca_sumatorios_comparados'].std()

resultado = f"{promedio_canad:.2f} ± {desviacion_estandar_canad:.2f}"
print("Mitjana ± Desviació Estàndard de la canadenca:", resultado)

Mitjana ± Desviació Estàndard de la canadenca: 7.00 ± nan


In [137]:
promedio_canad = CON_PA_MECVV['Canadenca_sumatorios_comparados'].mean()
desviacion_estandar_canad = CON_PA_MECVV['Canadenca_sumatorios_comparados'].std()

resultado = f"{promedio_canad:.2f} ± {desviacion_estandar_canad:.2f}"
print("Mitjana ± Desviació Estàndard de la canadenca:", resultado)

Mitjana ± Desviació Estàndard de la canadenca: 7.00 ± nan


In [138]:
promedio_canad = SIN_PA['Canadenca_sumatorios_comparados'].mean()
desviacion_estandar_canad = SIN_PA['Canadenca_sumatorios_comparados'].std()

resultado = f"{promedio_canad:.2f} ± {desviacion_estandar_canad:.2f}"
print("Mitjana ± Desviació Estàndard de la canadenca:", resultado)

Mitjana ± Desviació Estàndard de la canadenca: 8.00 ± 1.41


#### Pvalor

##### Normalitat

In [139]:
stat, p_value = shapiro(CON_PA['Canadenca_sumatorios_comparados']).dropna()
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

AttributeError: 'ShapiroResult' object has no attribute 'dropna'

In [140]:
stat, p_value = shapiro(CON_PA_MECVV['Canadenca_sumatorios_comparados']).dropna()
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

AttributeError: 'ShapiroResult' object has no attribute 'dropna'

In [141]:
stat, p_value = shapiro(SIN_PA['Canadenca_sumatorios_comparados']).dropna()
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

AttributeError: 'ShapiroResult' object has no attribute 'dropna'

#TODO: modificar esto si siguen normalidad

In [142]:
t_stat_12, p_val_12 = mannwhitneyu(CON_PA['Canadenca_sumatorios_comparados'], CON_PA_MECVV['Canadenca_sumatorios_comparados'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: nan


In [143]:
t_stat_13, p_val_13 = mannwhitneyu(CON_PA['Canadenca_sumatorios_comparados'], SIN_PA['Canadenca_sumatorios_comparados'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: nan


In [144]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['Canadenca_sumatorios_comparados'], SIN_PA['Canadenca_sumatorios_comparados'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: nan


### Valors canadenca

##### CON_PA 

In [145]:
mild_CON_PA = CON_PA['Canadenca_sumatorios_comparados'].between(10.5, 11.5)
Percentatge_mild_CON_PA = mild_CON_PA.mean() * 100
num_observaciones_mild_CON_PA = mild_CON_PA.sum()

print(f"Percentatge de mild_CON_PA: {Percentatge_mild_CON_PA:.2f}%")
print(f"Nombre d'observacions en mild_CON_PA: {num_observaciones_mild_CON_PA}")

Percentatge de mild_CON_PA: 0.00%
Nombre d'observacions en mild_CON_PA: 0


In [146]:
modneu_CON_PA = CON_PA['Canadenca_sumatorios_comparados'].between(5,10)
Percentatge_modneu_CON_PA = modneu_CON_PA.mean() * 100
num_observaciones_modneu_CON_PA = modneu_CON_PA.sum()

print(f"Percentatge de modneu_CON_PA: {Percentatge_modneu_CON_PA:.2f}%")
print(f"Nombre d'observacions en modneu_CON_PA: {num_observaciones_modneu_CON_PA}")

Percentatge de modneu_CON_PA: 5.88%
Nombre d'observacions en modneu_CON_PA: 1


In [147]:
sevneu_CON_PA = CON_PA['Canadenca_sumatorios_comparados'].between(1.5, 4.5)
Percentatge_sevneu_CON_PA = sevneu_CON_PA.mean() * 100
num_observaciones_sevneu_CON_PA = sevneu_CON_PA.sum()

print(f"Percentatge de sevneu_CON_PA: {Percentatge_sevneu_CON_PA:.2f}%")
print(f"Nombre d'observacions en sevneu_CON_PA: {num_observaciones_sevneu_CON_PA}")

Percentatge de sevneu_CON_PA: 0.00%
Nombre d'observacions en sevneu_CON_PA: 0


##### CON_PA_MECVV

In [148]:
mild_CON_PA_MECVV = CON_PA_MECVV['Canadenca_sumatorios_comparados'].between(10.5, 11.5)
Percentatge_mild_CON_PA_MECVV = mild_CON_PA_MECVV.mean() * 100
num_observaciones_mild_CON_PA_MECVV = mild_CON_PA_MECVV.sum()

print(f"Percentatge de mild_CON_PA_MECVV: {Percentatge_mild_CON_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en mild_CON_PA_MECVV: {num_observaciones_mild_CON_PA_MECVV}")

Percentatge de mild_CON_PA_MECVV: 0.00%
Nombre d'observacions en mild_CON_PA_MECVV: 0


In [149]:
modneu_CON_PA_MECVV = CON_PA_MECVV['Canadenca_sumatorios_comparados'].between(5,10)
Percentatge_modneu_CON_PA_MECVV = modneu_CON_PA_MECVV.mean() * 100
num_observaciones_modneu_CON_PA_MECVV = modneu_CON_PA_MECVV.sum()

print(f"Percentatge de modneu_CON_PA_MECVV: {Percentatge_modneu_CON_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en modneu_CON_PA_MECVV: {num_observaciones_modneu_CON_PA_MECVV}")

Percentatge de modneu_CON_PA_MECVV: 10.00%
Nombre d'observacions en modneu_CON_PA_MECVV: 1


In [150]:
sevneu_CON_PA_MECVV = CON_PA_MECVV['Canadenca_sumatorios_comparados'].between(1.5, 4.5)
Percentatge_sevneu_CON_PA_MECVV = sevneu_CON_PA_MECVV.mean() * 100
num_observaciones_sevneu_CON_PA_MECVV = sevneu_CON_PA_MECVV.sum()

print(f"Percentatge de sevneu_CON_PA_MECVV: {Percentatge_sevneu_CON_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en sevneu_CON_PA_MECVV: {num_observaciones_sevneu_CON_PA_MECVV}")

Percentatge de sevneu_CON_PA_MECVV: 0.00%
Nombre d'observacions en sevneu_CON_PA_MECVV: 0


##### SIN_PA

In [151]:
mild_SIN_PA = SIN_PA['Canadenca_sumatorios_comparados'].between(10.5, 11.5)
Percentatge_mild_SIN_PA = mild_SIN_PA.mean() * 100
num_observaciones_mild_SIN_PA = mild_SIN_PA.sum()

print(f"Percentatge de mild_SIN_PA: {Percentatge_mild_SIN_PA:.2f}%")
print(f"Nombre d'observacions en mild_SIN_PA: {num_observaciones_mild_SIN_PA}")

Percentatge de mild_SIN_PA: 0.00%
Nombre d'observacions en mild_SIN_PA: 0


In [152]:
modneu_SIN_PA = SIN_PA['Canadenca_sumatorios_comparados'].between(5,10)
Percentatge_modneu_SIN_PA = modneu_SIN_PA.mean() * 100
num_observaciones_modneu_SIN_PA = modneu_SIN_PA.sum()

print(f"Percentatge de modneu_SIN_PA: {Percentatge_modneu_SIN_PA:.2f}%")
print(f"Nombre d'observacions en modneu_SIN_PA: {num_observaciones_modneu_SIN_PA}")

Percentatge de modneu_SIN_PA: 4.08%
Nombre d'observacions en modneu_SIN_PA: 2


In [153]:
sevneu_SIN_PA = SIN_PA['Canadenca_sumatorios_comparados'].between(1.5, 4.5)
Percentatge_sevneu_SIN_PA = sevneu_SIN_PA.mean() * 100
num_observaciones_sevneu_SIN_PA = sevneu_SIN_PA.sum()

print(f"Percentatge de sevneu_SIN_PA: {Percentatge_sevneu_SIN_PA:.2f}%")
print(f"Nombre d'observacions en sevneu_SIN_PA: {num_observaciones_sevneu_SIN_PA}")

Percentatge de sevneu_SIN_PA: 0.00%
Nombre d'observacions en sevneu_SIN_PA: 0


#### Pvalor

##### Dèficit neurològic lleuger (mild)

In [154]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['Canadenca_sumatorios_comparados'].between(10.5,11.5), CON_PA_MECVV['Canadenca_sumatorios_comparados'].between(10.5,11.5), equal_var=False)
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: nan


In [155]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['Canadenca_sumatorios_comparados'].between(10.5,11.5), SIN_PA['Canadenca_sumatorios_comparados'].between(10.5,11.5), equal_var=False)
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: nan


In [156]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['Canadenca_sumatorios_comparados'].between(10.5,11.5), SIN_PA['Canadenca_sumatorios_comparados'].between(10.5,11.5), equal_var=False)
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: nan


##### Dèficit neurològic moderat

In [157]:
t_stat_12, p_val_12 = mannwhitneyu(CON_PA['Canadenca_sumatorios_comparados'].between(5,10), CON_PA_MECVV['Canadenca_sumatorios_comparados'].between(5, 10), alternative='less')
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: 0.3700051180291794


In [158]:
t_stat_13, p_val_13 = mannwhitneyu(CON_PA['Canadenca_sumatorios_comparados'].between(5,10), SIN_PA['Canadenca_sumatorios_comparados'].between(5, 10), alternative='less')
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: 0.6274490868971419


In [159]:
t_stat_23, p_val_23 = mannwhitneyu(CON_PA_MECVV['Canadenca_sumatorios_comparados'].between(5,10), SIN_PA['Canadenca_sumatorios_comparados'].between(5, 10), alternative='less')
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: 0.7870742763184819


##### Dèficit neurològic greu (sever)

In [160]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['Canadenca_sumatorios_comparados'].between(1.5,4.5), CON_PA_MECVV['Canadenca_sumatorios_comparados'].between(1.5,4.5), equal_var=False)
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: nan


In [161]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['Canadenca_sumatorios_comparados'].between(1.5, 4.5), SIN_PA['Canadenca_sumatorios_comparados'].between(1.5, 4.5), equal_var=False)
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: nan


In [162]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['Canadenca_sumatorios_comparados'].between(1.5,4.5), SIN_PA['Canadenca_sumatorios_comparados'].between(1.5, 4.5), equal_var=False)
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: nan


## Infeccions respiratòries

##### CON_PA 

In [163]:
mainresp_CON_PA= (CON_PA['Main_respiratory_infections_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de infeccions respiratòries principals : {mainresp_CON_PA:.2f}%")

Percentatge de infeccions respiratòries principals : 41.18%


In [164]:
LRTI_CON_PA= (CON_PA['LRTI_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de LRTI: {LRTI_CON_PA:.2f}%")

Percentatge de LRTI: 17.65%


In [165]:
COPD_CON_PA= (CON_PA['COPD_exacerbations_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de exacerbacions de COPD: {COPD_CON_PA:.2f}%")

Percentatge de exacerbacions de COPD: 29.41%


In [166]:
fibro_CON_PA= (CON_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de fibrosi pulmonar i fibrotòrax: {fibro_CON_PA:.2f}%")

Percentatge de fibrosi pulmonar i fibrotòrax: 0.00%


##### CON_PA_MECVV

In [167]:
mainresp_CON_PA_MECVV= (CON_PA_MECVV['Main_respiratory_infections_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de infeccions respiratòries principals : {mainresp_CON_PA_MECVV:.2f}%")

Percentatge de infeccions respiratòries principals : 50.00%


In [168]:
LRTI_CON_PA_MECVV= (CON_PA_MECVV['LRTI_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de LRTI: {LRTI_CON_PA_MECVV:.2f}%")

Percentatge de LRTI: 20.00%


In [169]:
COPD_CON_PA_MECVV = (CON_PA_MECVV['COPD_exacerbations_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de exacerbacions de COPD: {COPD_CON_PA_MECVV:.2f}%")

Percentatge de exacerbacions de COPD: 30.00%


In [170]:
fibro_CON_PA_MECVV = (CON_PA_MECVV['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de fibrosi pulmonar i fibrotòrax: {fibro_CON_PA_MECVV:.2f}%")

Percentatge de fibrosi pulmonar i fibrotòrax: 0.00%


##### SIN_PA

In [171]:
mainresp_SIN_PA= (SIN_PA['Main_respiratory_infections_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de infeccions respiratòries principals : {mainresp_SIN_PA:.2f}%")

Percentatge de infeccions respiratòries principals : 42.86%


In [172]:
LRTI_SIN_PA= (SIN_PA['LRTI_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de LRTI: {LRTI_SIN_PA:.2f}%")

Percentatge de LRTI: 22.45%


In [173]:
COPD_SIN_PA= (SIN_PA['COPD_exacerbations_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de exacerbacions de COPD: {COPD_SIN_PA:.2f}%")

Percentatge de exacerbacions de COPD: 32.65%


In [174]:
fibro_SIN_PA= (SIN_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de fibrosi pulmonar i fibrotòrax: {fibro_SIN_PA:.2f}%")

Percentatge de fibrosi pulmonar i fibrotòrax: 0.00%


#### Pvalor

##### Normalitat

In [175]:
stat, p_value = shapiro(CON_PA['Main_respiratory_infections_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.6317
Valor p = 0.00002144
El nombre de files no segueix una distribució normal.


In [176]:
stat, p_value = shapiro(CON_PA['LRTI_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.4702
Valor p = 0.00000073
El nombre de files no segueix una distribució normal.


In [177]:
stat, p_value = shapiro(CON_PA['COPD_exacerbations_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.5785
Valor p = 0.00000650
El nombre de files no segueix una distribució normal.


In [178]:
stat, p_value = shapiro(CON_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 1.0000
Valor p = 1.00000000
El nombre de files segueix una distribució normal.


C:\Users\maria\PycharmProjects\practicum\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


Usaré Mann-Whitney cuando tenga que comparar contra CON_PA

In [179]:
stat, p_value = shapiro(CON_PA_MECVV['Main_respiratory_infections_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.6553
Valor p = 0.00025396
El nombre de files no segueix una distribució normal.


In [180]:
stat, p_value = shapiro(CON_PA_MECVV['LRTI_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.5093
Valor p = 0.00000467
El nombre de files no segueix una distribució normal.


In [181]:
stat, p_value = shapiro(CON_PA_MECVV['COPD_exacerbations_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.5942
Valor p = 0.00004713
El nombre de files no segueix una distribució normal.


In [182]:
stat, p_value = shapiro(CON_PA_MECVV['Pulmonary_fibrosis_fibrotorax_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 1.0000
Valor p = 1.00000000
El nombre de files segueix una distribució normal.


Usaré también Mann-Whitney

In [183]:
stat, p_value = shapiro(SIN_PA['Main_respiratory_infections_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.6294
Valor p = 0.00000000
El nombre de files no segueix una distribució normal.


In [184]:
stat, p_value = shapiro(SIN_PA['LRTI_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.5158
Valor p = 0.00000000
El nombre de files no segueix una distribució normal.


In [185]:
stat, p_value = shapiro(SIN_PA['COPD_exacerbations_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.5916
Valor p = 0.00000000
El nombre de files no segueix una distribució normal.


In [186]:
stat, p_value = shapiro(SIN_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 1.0000
Valor p = 1.00000000
El nombre de files segueix una distribució normal.


##### Infeccions respiratòries principals 

In [187]:
stat_12, p_val_12 = mannwhitneyu(CON_PA['Main_respiratory_infections_diagnosticada'] == 1.0, CON_PA_MECVV['Main_respiratory_infections_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: 0.6832089509594992


In [188]:
stat_13, p_val_13 = mannwhitneyu(CON_PA['Main_respiratory_infections_diagnosticada'] == 1.0, SIN_PA['Main_respiratory_infections_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: 0.9113557832989896


In [189]:
stat_23, p_val_23 = mannwhitneyu(CON_PA_MECVV['Main_respiratory_infections_diagnosticada'] == 1.0, SIN_PA['Main_respiratory_infections_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: 0.6896369959432476


##### LRTI

In [190]:
stat_12, p_val_12 = mannwhitneyu(CON_PA['LRTI_diagnosticada'] == 1.0, CON_PA_MECVV['LRTI_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: 0.9109318987037512


In [191]:
stat_13, p_val_13 = mannwhitneyu(CON_PA['LRTI_diagnosticada'] == 1.0, SIN_PA['LRTI_diagnosticada'] == 1.0,alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: 0.6863825106164784


In [192]:
stat_23, p_val_23 = mannwhitneyu(CON_PA_MECVV['LRTI_diagnosticada'] == 1.0, SIN_PA['LRTI_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: 0.8770107151211659


##### Exacerbacions de COPD

In [193]:
stat_12, p_val_12 = mannwhitneyu(CON_PA['COPD_exacerbations_diagnosticada'] == 1.0, CON_PA_MECVV['COPD_exacerbations_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: 1.0


In [194]:
stat_13, p_val_13 = mannwhitneyu(CON_PA['COPD_exacerbations_diagnosticada'] == 1.0, SIN_PA['COPD_exacerbations_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: 0.8132303236327403


In [195]:
stat_23, p_val_23 = mannwhitneyu(CON_PA_MECVV['COPD_exacerbations_diagnosticada'] == 1.0, SIN_PA['COPD_exacerbations_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: 0.8809554446418177


##### Fibrosi pulmonar i fibrotòrax

In [196]:
stat_12, p_val_12 = mannwhitneyu(CON_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, CON_PA_MECVV['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: 1.0


In [197]:
stat_13, p_val_13 = mannwhitneyu(CON_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, SIN_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: 1.0


In [198]:
stat_23, p_val_23 = mannwhitneyu(CON_PA_MECVV['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, SIN_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: 1.0


## Resultats de laboratori

Les columnes generades a partir d'aquests resultats contenen valors tipus objecte, per tant, primer els transformaré a tipus numèric per poder realitzar les operacions que neccessitaré fer.

### Albúmina

In [199]:
CON_PA.loc[:, 'albumina'] = pd.to_numeric(CON_PA['albumina'], errors='coerce')

promedio_alb = CON_PA['albumina'].mean()
desviacion_estandar_alb = CON_PA['albumina'].std()

resultado = f"{promedio_alb:.2f} ± {desviacion_estandar_alb:.2f}"
print("Mitjana ± Desviació Estàndard de la albumina:", resultado)

Mitjana ± Desviació Estàndard de la albumina: 3.34 ± 0.38


In [200]:
CON_PA_MECVV.loc[:, 'albumina'] = pd.to_numeric(CON_PA_MECVV['albumina'], errors='coerce')

promedio_alb = CON_PA_MECVV['albumina'].mean()
desviacion_estandar_alb = CON_PA_MECVV['albumina'].std()

resultado = f"{promedio_alb:.2f} ± {desviacion_estandar_alb:.2f}"
print("Mitjana ± Desviació Estàndard de la albumina:", resultado)

Mitjana ± Desviació Estàndard de la albumina: 3.47 ± 0.55


In [201]:
SIN_PA.loc[:, 'albumina'] = pd.to_numeric(SIN_PA['albumina'], errors='coerce')

promedio_alb = SIN_PA['albumina'].mean()
desviacion_estandar_alb = SIN_PA['albumina'].std()

resultado = f"{promedio_alb:.2f} ± {desviacion_estandar_alb:.2f}"
print("Mitjana ± Desviació Estàndard de la albumina:", resultado)

Mitjana ± Desviació Estàndard de la albumina: 3.58 ± 0.64


#### Pvalor

##### Normalitat

In [202]:
albpa = CON_PA['albumina'].dropna()
stat, p_value = shapiro(albpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9311
Valor p = 0.56068160
El nombre de files segueix una distribució normal.


In [203]:
albconpamecvv = CON_PA_MECVV['albumina'].dropna()
stat, p_value = shapiro(albconpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9973
Valor p = 0.89985028
El nombre de files segueix una distribució normal.


In [204]:
albsinpa= SIN_PA['albumina'].dropna()
stat, p_value = shapiro(albsinpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9778
Valor p = 0.92442182
El nombre de files segueix una distribució normal.


In [205]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['albumina'], CON_PA_MECVV['albumina'], equal_var=False)
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: nan


In [206]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['albumina'], SIN_PA['albumina'], equal_var=False)
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: nan


In [207]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['albumina'], SIN_PA['albumina'], equal_var=False)
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: nan


### Proteïnes totals

In [208]:
CON_PA.loc[:, 'proteinas totales'] = pd.to_numeric(CON_PA['proteinas totales'], errors='coerce')

promedio_prot = CON_PA['proteinas totales'].mean()
desviacion_estandar_prot = CON_PA['proteinas totales'].std()

resultado = f"{promedio_prot:.2f} ± {desviacion_estandar_prot:.2f}"
print("Mitjana ± Desviació Estàndard de las proteinas totales:", resultado)

Mitjana ± Desviació Estàndard de las proteinas totales: 6.03 ± 0.73


In [209]:
CON_PA_MECVV.loc[:, 'proteinas totales'] = pd.to_numeric(CON_PA_MECVV['proteinas totales'], errors='coerce')

promedio_prot = CON_PA_MECVV['proteinas totales'].mean()
desviacion_estandar_prot = CON_PA_MECVV['proteinas totales'].std()

resultado = f"{promedio_prot:.2f} ± {desviacion_estandar_prot:.2f}"
print("Mitjana ± Desviació Estàndard de las proteinas totales:", resultado)

Mitjana ± Desviació Estàndard de las proteinas totales: 6.47 ± 0.76


In [210]:
SIN_PA.loc[:, 'proteinas totales'] = pd.to_numeric(SIN_PA['proteinas totales'], errors='coerce')

promedio_prot = SIN_PA['proteinas totales'].mean()
desviacion_estandar_prot = SIN_PA['proteinas totales'].std()

resultado = f"{promedio_prot:.2f} ± {desviacion_estandar_prot:.2f}"
print("Mitjana ± Desviació Estàndard de las proteinas totales:", resultado)

Mitjana ± Desviació Estàndard de las proteinas totales: 6.12 ± 0.72


#### Pvalor

##### Normalitat

In [211]:
protconpa=CON_PA['proteinas totales'].dropna()
stat, p_value = shapiro(protconpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9780
Valor p = 0.95346065
El nombre de files segueix una distribució normal.


In [212]:
protconpamecvv=CON_PA_MECVV['proteinas totales'].dropna()
stat, p_value = shapiro(protconpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9975
Valor p = 0.99159177
El nombre de files segueix una distribució normal.


In [213]:
protsinpa=SIN_PA['proteinas totales'].dropna()
stat, p_value = shapiro(protsinpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9753
Valor p = 0.86021925
El nombre de files segueix una distribució normal.


In [214]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['proteinas totales'], CON_PA_MECVV['proteinas totales'], equal_var=False)
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: nan


In [215]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['proteinas totales'], SIN_PA['proteinas totales'], equal_var=False)
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: nan


In [216]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['proteinas totales'], SIN_PA['proteinas totales'], equal_var=False)
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: nan


### Hemoglobina

In [217]:
CON_PA.loc[:, 'Hb'] = pd.to_numeric(CON_PA['Hb'], errors='coerce')

promedio_hb = CON_PA['Hb'].mean()
desviacion_estandar_hb = CON_PA['Hb'].std()

resultado = f"{promedio_hb:.2f} ± {desviacion_estandar_hb:.2f}"
print("Mitjana ± Desviació Estàndard de la hemoglobina:", resultado)

Mitjana ± Desviació Estàndard de la hemoglobina: 12.34 ± 1.77


In [218]:
CON_PA_MECVV.loc[:, 'Hb'] = pd.to_numeric(CON_PA_MECVV['Hb'], errors='coerce')

promedio_hb = CON_PA_MECVV['Hb'].mean()
desviacion_estandar_hb = CON_PA_MECVV['Hb'].std()

resultado = f"{promedio_hb:.2f} ± {desviacion_estandar_hb:.2f}"
print("Mitjana ± Desviació Estàndard de la hemoglobina:", resultado)

Mitjana ± Desviació Estàndard de la hemoglobina: 13.22 ± 1.40


In [219]:
SIN_PA.loc[:, 'Hb'] = pd.to_numeric(SIN_PA['Hb'], errors='coerce')

promedio_hb = SIN_PA['Hb'].mean()
desviacion_estandar_hb = SIN_PA['Hb'].std()

resultado = f"{promedio_hb:.2f} ± {desviacion_estandar_hb:.2f}"
print("Mitjana ± Desviació Estàndard de la hemoglobina:", resultado)

Mitjana ± Desviació Estàndard de la hemoglobina: 13.22 ± 2.16


##### Pvalor

##### Normalitat

In [220]:
hbconpa=CON_PA['Hb'].dropna()
stat, p_value = shapiro(hbconpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9304
Valor p = 0.48501101
El nombre de files segueix una distribució normal.


In [221]:
hbconpamecvv=CON_PA_MECVV['Hb'].dropna()
stat, p_value = shapiro(hbconpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8592
Valor p = 0.25743995
El nombre de files segueix una distribució normal.


In [222]:
hbsinpa=SIN_PA['Hb'].dropna()
stat, p_value = shapiro(hbsinpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9551
Valor p = 0.37239632
El nombre de files segueix una distribució normal.


In [223]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['Hb'], CON_PA_MECVV['Hb'], equal_var=False)
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: nan


In [224]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['Hb'], SIN_PA['Hb'], equal_var=False)
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: nan


In [225]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['Hb'], SIN_PA['Hb'], equal_var=False)
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: nan


### Colesterol 

In [226]:
CON_PA.loc[:, 'colesterol total'] = pd.to_numeric(CON_PA['colesterol total'], errors='coerce')

promedio_col = CON_PA['colesterol total'].mean()
desviacion_estandar_col = CON_PA['colesterol total'].std()

resultado = f"{promedio_col:.2f} ± {desviacion_estandar_col:.2f}"
print("Mitjana ± Desviació Estàndard del colesterol total:", resultado)

Mitjana ± Desviació Estàndard del colesterol total: 132.10 ± 34.83


In [227]:
CON_PA_MECVV.loc[:, 'colesterol total'] = pd.to_numeric(CON_PA_MECVV['colesterol total'], errors='coerce')

promedio_col = CON_PA_MECVV['colesterol total'].mean()
desviacion_estandar_col = CON_PA_MECVV['colesterol total'].std()

resultado = f"{promedio_col:.2f} ± {desviacion_estandar_col:.2f}"
print("Mitjana ± Desviació Estàndard del colesterol total:", resultado)

Mitjana ± Desviació Estàndard del colesterol total: 111.90 ± 28.37


In [228]:
SIN_PA.loc[:, 'colesterol total'] = pd.to_numeric(SIN_PA['colesterol total'], errors='coerce')

promedio_col = SIN_PA['colesterol total'].mean()
desviacion_estandar_col = SIN_PA['colesterol total'].std()

resultado = f"{promedio_col:.2f} ± {desviacion_estandar_col:.2f}"
print("Mitjana ± Desviació Estàndard del colesterol total:", resultado)

Mitjana ± Desviació Estàndard del colesterol total: 145.27 ± 40.45


#### Pvalor

##### Normalitat 



In [229]:
colconpa=CON_PA['colesterol total'].dropna()
stat, p_value = shapiro(colconpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9887
Valor p = 0.99048226
El nombre de files segueix una distribució normal.


In [230]:
colconpamecvv=CON_PA_MECVV['colesterol total'].dropna()
stat, p_value = shapiro(colconpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9462
Valor p = 0.55290474
El nombre de files segueix una distribució normal.


In [231]:
colsinpa=SIN_PA['colesterol total'].dropna()
stat, p_value = shapiro(colsinpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9725
Valor p = 0.84373822
El nombre de files segueix una distribució normal.


In [232]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['colesterol total'], CON_PA_MECVV['colesterol total'], equal_var=False)
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: nan


In [233]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['colesterol total'], SIN_PA['colesterol total'], equal_var=False)
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: nan


In [234]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['colesterol total'], SIN_PA['colesterol total'], equal_var=False)
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: nan


### Leucòcits

In [235]:
CON_PA.loc[:, 'leucos'] = pd.to_numeric(CON_PA['leucos'], errors='coerce')

promedio_leu = CON_PA['leucos'].mean()
desviacion_estandar_leu = CON_PA['leucos'].std()

resultado = f"{promedio_leu:.2f} ± {desviacion_estandar_leu:.2f}"
print("Mitjana ± Desviació Estàndard dels leucòcits:", resultado)

Mitjana ± Desviació Estàndard dels leucòcits: 12.69 ± 4.88


In [236]:
CON_PA_MECVV.loc[:, 'leucos'] = pd.to_numeric(CON_PA_MECVV['leucos'], errors='coerce')

promedio_leu = CON_PA_MECVV['leucos'].mean()
desviacion_estandar_leu = CON_PA_MECVV['leucos'].std()

resultado = f"{promedio_leu:.2f} ± {desviacion_estandar_leu:.2f}"
print("Mitjana ± Desviació Estàndard dels leucòcits:", resultado)

Mitjana ± Desviació Estàndard dels leucòcits: 13.08 ± 6.83


In [237]:
SIN_PA.loc[:, 'leucos'] = pd.to_numeric(SIN_PA['leucos'], errors='coerce')

promedio_leu = SIN_PA['leucos'].mean()
desviacion_estandar_leu = SIN_PA['leucos'].std()

resultado = f"{promedio_leu:.2f} ± {desviacion_estandar_leu:.2f}"
print("Mitjana ± Desviació Estàndard dels leucòcits:", resultado)

Mitjana ± Desviació Estàndard dels leucòcits: 11.52 ± 6.52


#### Pvalor 

##### Normalitat

In [238]:
leuconpa=CON_PA['leucos'].dropna()
stat, p_value = shapiro(leuconpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8482
Valor p = 0.09127190
El nombre de files segueix una distribució normal.


In [239]:
leuconpamecvv=CON_PA_MECVV['leucos'].dropna()
stat, p_value = shapiro(leuconpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8939
Valor p = 0.40148390
El nombre de files segueix una distribució normal.


In [240]:
leusinpa=SIN_PA['leucos'].dropna()
stat, p_value = shapiro(leusinpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8178
Valor p = 0.00096236
El nombre de files no segueix una distribució normal.


In [241]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['leucos'], CON_PA_MECVV['leucos'], equal_var=False)
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: nan


In [242]:
t_stat_13, p_val_13 = mannwhitneyu(CON_PA['leucos'], SIN_PA['leucos'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: nan


In [243]:
t_stat_23, p_val_23 = mannwhitneyu(CON_PA_MECVV['leucos'], SIN_PA['leucos'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: nan


### Limfòcits %

In [244]:
CON_PA.loc[:, 'limfos'] = pd.to_numeric(CON_PA['limfos'], errors='coerce')

promedio_limf = CON_PA['limfos'].mean()
desviacion_estandar_limf = CON_PA['limfos'].std()

resultado = f"{promedio_limf:.2f} ± {desviacion_estandar_limf:.2f}"
print("Mitjana ± Desviació Estàndard dels limfòcits %:", resultado)

Mitjana ± Desviació Estàndard dels limfòcits %: 16.70 ± 12.29


In [245]:
CON_PA_MECVV.loc[:, 'limfos'] = pd.to_numeric(CON_PA_MECVV['limfos'], errors='coerce')

promedio_limf = CON_PA_MECVV['limfos'].mean()
desviacion_estandar_limf = CON_PA_MECVV['limfos'].std()

resultado = f"{promedio_limf:.2f} ± {desviacion_estandar_limf:.2f}"
print("Mitjana ± Desviació Estàndard dels limfòcits %:", resultado)

Mitjana ± Desviació Estàndard dels limfòcits %: 17.20 ± 9.35


In [246]:
SIN_PA.loc[:, 'limfos'] = pd.to_numeric(SIN_PA['limfos'], errors='coerce')

promedio_limf = SIN_PA['limfos'].mean()
desviacion_estandar_limf = SIN_PA['limfos'].std()

resultado = f"{promedio_limf:.2f} ± {desviacion_estandar_limf:.2f}"
print("Mitjana ± Desviació Estàndard dels limfòcits %:", resultado)

Mitjana ± Desviació Estàndard dels limfòcits %: 19.50 ± 11.57


#### Pvalor

##### Normalitat

In [247]:
limfconpa=CON_PA['limfos'].dropna()
stat, p_value = shapiro(limfconpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8208
Valor p = 0.04757269
El nombre de files no segueix una distribució normal.


In [248]:
limfconpamecvv=CON_PA_MECVV['limfos'].dropna()
stat, p_value = shapiro(limfconpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9374
Valor p = 0.51710593
El nombre de files segueix una distribució normal.


In [249]:
limfsinpa=SIN_PA['limfos'].dropna()
stat, p_value = shapiro(limfsinpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9290
Valor p = 0.13154504
El nombre de files segueix una distribució normal.


In [250]:
stat_12, p_val_12 = mannwhitneyu(CON_PA['limfos'], CON_PA_MECVV['limfos'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: nan


In [251]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['limfos'], SIN_PA['limfos'], equal_var=False)
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: nan


In [252]:
stat_23, p_val_23 = mannwhitneyu(CON_PA_MECVV['limfos'], SIN_PA['limfos'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: nan


### Proteïna C Reactiva

In [253]:
CON_PA.loc[:, 'prot C react'] = pd.to_numeric(CON_PA['prot C react'], errors='coerce')

promedio_protcr = CON_PA['prot C react'].mean()
desviacion_estandar_protcr = CON_PA['prot C react'].std()

resultado = f"{promedio_protcr:.2f} ± {desviacion_estandar_protcr:.2f}"
print("Mitjana ± Desviació Estàndard de la proteïna C reactiva:", resultado)

Mitjana ± Desviació Estàndard de la proteïna C reactiva: 8.90 ± 5.47


In [254]:
CON_PA_MECVV.loc[:, 'prot C react'] = pd.to_numeric(CON_PA_MECVV['prot C react'], errors='coerce')

promedio_protcr = CON_PA_MECVV['prot C react'].mean()
desviacion_estandar_protcr = CON_PA_MECVV['prot C react'].std()

resultado = f"{promedio_protcr:.2f} ± {desviacion_estandar_protcr:.2f}"
print("Mitjana ± Desviació Estàndard de la proteïna C reactiva:", resultado)

Mitjana ± Desviació Estàndard de la proteïna C reactiva: 8.88 ± 3.63


In [255]:
SIN_PA.loc[:, 'prot C react'] = pd.to_numeric(SIN_PA['prot C react'], errors='coerce')

promedio_protcr = SIN_PA['prot C react'].mean()
desviacion_estandar_protcr = SIN_PA['prot C react'].std()

resultado = f"{promedio_protcr:.2f} ± {desviacion_estandar_protcr:.2f}"
print("Mitjana ± Desviació Estàndard de la proteïna C reactiva:", resultado)

Mitjana ± Desviació Estàndard de la proteïna C reactiva: 6.16 ± 6.19


#### Pvalor

##### Normalitat

In [256]:
pcrconpa=CON_PA['prot C react'].dropna()
stat, p_value = shapiro(pcrconpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9423
Valor p = 0.60665876
El nombre de files segueix una distribució normal.


In [257]:
pcrconpamecvv=CON_PA_MECVV['prot C react'].dropna()
stat, p_value = shapiro(pcrconpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.7886
Valor p = 0.08325234
El nombre de files segueix una distribució normal.


In [258]:
pcrsinpa=SIN_PA['prot C react'].dropna()
stat, p_value = shapiro(pcrsinpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8698
Valor p = 0.00636559
El nombre de files no segueix una distribució normal.


In [259]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['prot C react'], CON_PA_MECVV['prot C react'], equal_var=False)
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: nan


In [260]:
t_stat_13, p_val_13 = mannwhitneyu(CON_PA['prot C react'], SIN_PA['prot C react'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: nan


In [261]:
t_stat_23, p_val_23 = mannwhitneyu(CON_PA_MECVV['prot C react'], SIN_PA['prot C react'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: nan


### Urea 

In [262]:
CON_PA.loc[:, 'urea'] = pd.to_numeric(CON_PA['urea'], errors='coerce')

promedio_urea = CON_PA['urea'].mean()
desviacion_estandar_urea = CON_PA['urea'].std()

resultado = f"{promedio_urea:.2f} ± {desviacion_estandar_urea:.2f}"
print("Mitjana ± Desviació Estàndard de la urea:", resultado)

Mitjana ± Desviació Estàndard de la urea: 62.17 ± 36.37


In [263]:
CON_PA_MECVV.loc[:, 'urea'] = pd.to_numeric(CON_PA_MECVV['urea'], errors='coerce')

promedio_urea = CON_PA_MECVV['urea'].mean()
desviacion_estandar_urea = CON_PA_MECVV['urea'].std()

resultado = f"{promedio_urea:.2f} ± {desviacion_estandar_urea:.2f}"
print("Mitjana ± Desviació Estàndard de la urea:", resultado)

Mitjana ± Desviació Estàndard de la urea: 62.37 ± 37.72


In [264]:
SIN_PA.loc[:, 'urea'] = pd.to_numeric(SIN_PA['urea'], errors='coerce')

promedio_urea = SIN_PA['urea'].mean()
desviacion_estandar_urea = SIN_PA['urea'].std()

resultado = f"{promedio_urea:.2f} ± {desviacion_estandar_urea:.2f}"
print("Mitjana ± Desviació Estàndard de la urea:", resultado)

Mitjana ± Desviació Estàndard de la urea: 55.03 ± 30.66


#### Pvalor

##### Normalitat

In [265]:
uconpa=CON_PA['urea'].dropna()
stat, p_value = shapiro(uconpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8842
Valor p = 0.17374314
El nombre de files segueix una distribució normal.


In [266]:
uconpamecvv=CON_PA_MECVV['urea'].dropna()
stat, p_value = shapiro(uconpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8991
Valor p = 0.42677381
El nombre de files segueix una distribució normal.


In [267]:
usinpa=SIN_PA['urea'].dropna()
stat, p_value = shapiro(usinpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8289
Valor p = 0.00115364
El nombre de files no segueix una distribució normal.


In [268]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['urea'], CON_PA_MECVV['urea'], equal_var=False)
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: nan


In [269]:
t_stat_13, p_val_13 = mannwhitneyu(CON_PA['urea'], SIN_PA['urea'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: nan


In [270]:
t_stat_23, p_val_23 = mannwhitneyu(CON_PA_MECVV['urea'], SIN_PA['urea'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: nan


### Filtrat glomerular CDK-EPI

In [271]:
CON_PA.loc[:, 'FGE CDK-EPI'] = pd.to_numeric(CON_PA['FGE CDK-EPI'], errors='coerce')

promedio_fg = CON_PA['FGE CDK-EPI'].mean()
desviacion_estandar_fg = CON_PA['FGE CDK-EPI'].std()

resultado = f"{promedio_fg:.2f} ± {desviacion_estandar_fg:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI:", resultado)

Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI: 56.00 ± 12.65


In [272]:
CON_PA_MECVV.loc[:, 'FGE CDK-EPI'] = pd.to_numeric(CON_PA_MECVV['FGE CDK-EPI'], errors='coerce')

promedio_fg = CON_PA_MECVV['FGE CDK-EPI'].mean()
desviacion_estandar_fg = CON_PA_MECVV['FGE CDK-EPI'].std()

resultado = f"{promedio_fg:.2f} ± {desviacion_estandar_fg:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI:", resultado)

Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI: 54.67 ± 13.32


In [273]:
SIN_PA.loc[:, 'FGE CDK-EPI'] = pd.to_numeric(SIN_PA['FGE CDK-EPI'], errors='coerce')

promedio_fg = SIN_PA['FGE CDK-EPI'].mean()
desviacion_estandar_fg = SIN_PA['FGE CDK-EPI'].std()

resultado = f"{promedio_fg:.2f} ± {desviacion_estandar_fg:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI:", resultado)

Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI: 58.56 ± 18.40


#### Pvalor

##### Normalitat

In [274]:
fgepiconpa=CON_PA['FGE CDK-EPI'].dropna()
stat, p_value = shapiro(fgepiconpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.7690
Valor p = 0.01994941
El nombre de files no segueix una distribució normal.


In [275]:
fgepiconpamecvv=CON_PA_MECVV['FGE CDK-EPI'].dropna()
stat, p_value = shapiro(fgepiconpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8120
Valor p = 0.14355397
El nombre de files segueix una distribució normal.


In [276]:
fgepisinpa=SIN_PA['FGE CDK-EPI'].dropna()
stat, p_value = shapiro(fgepisinpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9160
Valor p = 0.14543070
El nombre de files segueix una distribució normal.


In [277]:
t_stat_12, p_val_12 = mannwhitneyu(CON_PA['FGE CDK-EPI'], CON_PA_MECVV['FGE CDK-EPI'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: nan


In [278]:
t_stat_13, p_val_13 = mannwhitneyu(CON_PA['FGE CDK-EPI'], SIN_PA['FGE CDK-EPI'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_13)

P-valor entre CON_PA i CON_PA_MECVV: nan


In [279]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['FGE CDK-EPI'], SIN_PA['FGE CDK-EPI'], equal_var=False)
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: nan


### Filtrat glomerular MDRD

In [280]:
CON_PA.loc[:, 'FGE MDRD'] = pd.to_numeric(CON_PA['FGE MDRD'], errors='coerce')

promedio_fgmd = CON_PA['FGE MDRD'].mean()
desviacion_estandar_fgmd = CON_PA['FGE MDRD'].std()

resultado = f"{promedio_fgmd:.2f} ± {desviacion_estandar_fgmd:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular MDRD:", resultado)

Mitjana ± Desviació Estàndard de la filtració glomerular MDRD: 51.50 ± 0.71


In [281]:
CON_PA_MECVV.loc[:, 'FGE MDRD'] = pd.to_numeric(CON_PA_MECVV['FGE MDRD'], errors='coerce')

promedio_fgmd = CON_PA_MECVV['FGE MDRD'].mean()
desviacion_estandar_fgmd = CON_PA_MECVV['FGE MDRD'].std()

resultado = f"{promedio_fgmd:.2f} ± {desviacion_estandar_fgmd:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular MDRD:", resultado)

Mitjana ± Desviació Estàndard de la filtració glomerular MDRD: nan ± nan


In [282]:
SIN_PA.loc[:, 'FGE MDRD'] = pd.to_numeric(SIN_PA['FGE MDRD'], errors='coerce')

promedio_fgmd = SIN_PA['FGE MDRD'].mean()
desviacion_estandar_fgmd = SIN_PA['FGE MDRD'].std()

resultado = f"{promedio_fgmd:.2f} ± {desviacion_estandar_fgmd:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular MDRD:", resultado)

Mitjana ± Desviació Estàndard de la filtració glomerular MDRD: 37.67 ± 17.43


#### Pvalor

##### Normalitat

In [283]:
fgmd = CON_PA['FGE MDRD'].dropna()
stat, p_value = shapiro(fgmd)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")


ValueError: Data must be at least length 3.

In [284]:
stat, p_value = shapiro(CON_PA_MECVV['FGE MDRD'].dropna())
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

ValueError: Data must be at least length 3.

In [285]:
stat, p_value = shapiro(SIN_PA['FGE MDRD'].dropna())
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8324
Valor p = 0.11252528
El nombre de files segueix una distribució normal.


In [286]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['FGE MDRD'], CON_PA_MECVV['FGE MDRD'], equal_var=False)
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: nan


In [287]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['FGE MDRD'], SIN_PA['FGE MDRD'], equal_var=False)
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_13)

P-valor entre CON_PA i CON_PA_MECVV: nan


In [288]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['FGE MDRD'], SIN_PA['FGE MDRD'], equal_var=False)
print("P-valor entre CON_PA_MECVV i CON_PA_MECVV:", p_val_23)

P-valor entre CON_PA_MECVV i CON_PA_MECVV: nan
